# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [112]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

In [113]:
sqrt_sum = 0
for sqrt_value in generator:
    print(sqrt_value)
    sqrt_sum += sqrt_value
print(sqrt_sum)

1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979
8.382332347441762


In [114]:
generator_2 = square_root_generator(13)
for val in generator_2:
  print(val)

1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979
2.449489742783178
2.6457513110645907
2.8284271247461903
3.0
3.1622776601683795
3.3166247903554
3.4641016151377544
3.605551275463989


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [115]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)

def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}

for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [116]:
#Install the dependencies
%%capture
!pip install dlt[duckdb]

In [117]:
import dlt
import duckdb

In [118]:
pipeline = dlt.pipeline(pipeline_name ='homework',destination='duckdb',dataset_name='people_details')

## Question 3

In [119]:
info = pipeline.run(people_1(), table_name="people", write_disposition ="replace")

info = pipeline.run(people_2(), table_name="people", write_disposition='append')

In [120]:
conn = duckdb.connect("homework.duckdb")

In [121]:
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
display(conn.sql("show tables"))

┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
└─────────────────────┘

In [122]:
ppl_after_append = conn.sql("SELECT * FROM people")
display(ppl_after_append)

┌───────┬──────────┬───────┬─────────┬────────────────────┬────────────────┬────────────┐
│  id   │   name   │  age  │  city   │    _dlt_load_id    │    _dlt_id     │ occupation │
│ int64 │ varchar  │ int64 │ varchar │      varchar       │    varchar     │  varchar   │
├───────┼──────────┼───────┼─────────┼────────────────────┼────────────────┼────────────┤
│     1 │ Person_1 │    26 │ City_A  │ 1707923020.2148511 │ wHCXdT8En203pA │ NULL       │
│     2 │ Person_2 │    27 │ City_A  │ 1707923020.2148511 │ FeiKdYY3jDKerQ │ NULL       │
│     3 │ Person_3 │    28 │ City_A  │ 1707923020.2148511 │ OTFxzLodAVDM5A │ NULL       │
│     4 │ Person_4 │    29 │ City_A  │ 1707923020.2148511 │ YYd7oStpFpoRYQ │ NULL       │
│     5 │ Person_5 │    30 │ City_A  │ 1707923020.2148511 │ l8h6apKjBHT/zQ │ NULL       │
│     3 │ Person_3 │    33 │ City_B  │ 1707923020.8285563 │ HvE5OND5HiYE3w │ Job_3      │
│     4 │ Person_4 │    34 │ City_B  │ 1707923020.8285563 │ m9HnbzT/JS23Gw │ Job_4      │
│     5 │ 

In [123]:
age_sum = conn.sql("SELECT SUM(age) FROM people")
display(age_sum)

┌──────────┐
│ sum(age) │
│  int128  │
├──────────┤
│      353 │
└──────────┘

## Question 4

In [128]:
info = pipeline.run(people_1(), table_name="people", write_disposition ="replace")

info = pipeline.run(people_2(), table_name="people", write_disposition='merge', primary_key='id')

In [129]:
conn.sql("show tables")

┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
└─────────────────────┘

In [130]:
ppl_after_merge = conn.sql("SELECT * FROM people")
display(ppl_after_merge)

┌───────┬──────────┬───────┬─────────┬────────────────────┬────────────────┬────────────┐
│  id   │   name   │  age  │  city   │    _dlt_load_id    │    _dlt_id     │ occupation │
│ int64 │ varchar  │ int64 │ varchar │      varchar       │    varchar     │  varchar   │
├───────┼──────────┼───────┼─────────┼────────────────────┼────────────────┼────────────┤
│     1 │ Person_1 │    26 │ City_A  │ 1707923220.9661536 │ uxNj2g2euZlfGA │ NULL       │
│     2 │ Person_2 │    27 │ City_A  │ 1707923220.9661536 │ uNND07UETHJLXQ │ NULL       │
│     5 │ Person_5 │    35 │ City_B  │ 1707923221.5160968 │ lHCez3NlOPJI2w │ Job_5      │
│     7 │ Person_7 │    37 │ City_B  │ 1707923221.5160968 │ WGfM9oadM2j8Ww │ Job_7      │
│     3 │ Person_3 │    33 │ City_B  │ 1707923221.5160968 │ cZFb7J/HCNmudg │ Job_3      │
│     6 │ Person_6 │    36 │ City_B  │ 1707923221.5160968 │ dkW5aq12AV0TdQ │ Job_6      │
│     8 │ Person_8 │    38 │ City_B  │ 1707923221.5160968 │ jqdh0G87nn+KmQ │ Job_8      │
│     4 │ 

In [131]:
age_sum = conn.sql("SELECT SUM(age) FROM people")
display(age_sum)

┌──────────┐
│ sum(age) │
│  int128  │
├──────────┤
│      266 │
└──────────┘

In [132]:
conn.close()

Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX